In [1]:
from bo_functions import LHS_Design
from bo_functions import create_y_data
from bo_functions import calc_y_exp
from bo_functions import test_train_split
from bo_functions import create_sse_data
from scipy.stats import qmc
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = False
seed = 9
LHS_Points =20
sampler = qmc.LatinHypercube(d=2, seed = seed)
sample = sampler.random(n=LHS_Points)
l_bounds =  [-2, -2]
u_bounds =  [2, 2]
LHS = qmc.scale(sample, l_bounds, u_bounds)
# print(LHS)

In [3]:
#Creates Experimental data and saves to a csv
Theta_True = np.array([1,-1])
Xexp = np.array([-2,-1,0,1,2])
n = len(Xexp)
noise_std = 0.1**2
d = 2
Yexp = calc_y_exp(Theta_True, Xexp, noise_std)

Exp_data = np.column_stack((Xexp,Yexp))

df = pd.DataFrame(Exp_data, columns = ["Xexp","Yexp"])

path = '../Input_CSVs/Exp_Data/d='+str(d)
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

In [4]:
all_p = []

#Loop over # of Theta Points
for i in range(len(LHS)):
    #Loop over # of state points
    if emulator == True:
        for j in Xexp:
            #Ensure correct index will be used
            index = int(i%LHS_Points + np.floor(i/LHS_Points))
            #Append data point to list
            all_p.append([LHS[index,0],LHS[index,1],j])
    else:
        all_p.append([LHS[i,0],LHS[i,1], 0]) #Note, zero is a placeholder and doesn't matter

#Turn all_p into an array
all_p = np.array([all_p])[0]
# print(all_p)
# print(all_p[0:5])
# print(all_p[5:10])
# print(all_p)


In [5]:
#Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")
t = str(len(all_p))

if emulator == False:
    all_2_in_y = create_sse_data(d,all_p,Xexp,Yexp)
    all_2_in_y_ln_obj = create_sse_data(d,all_p,Xexp,Yexp, obj = "LN_obj")
    all_2_data = np.column_stack((all_p, all_2_in_y))
    all_2_data_ln_obj = np.column_stack((all_p, all_2_in_y_ln_obj))
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    all_array = np.array([all_2_data,all_2_data_ln_obj])

else:
    all_3_in_y = create_y_data(all_p)
    all_3_data = np.column_stack((all_p, all_3_in_y))
    titles_all = np.array(['all_emul_data'])
    all_array = np.array([all_3_data])

#Save Train Data/Test Data
for i in range(len(titles_all)):
    df_all = pd.DataFrame(all_array[i], columns = ["Theta_1","Theta_2","X Coord","y-value"])
    path = '../Input_CSVs/Train_Data/d='+str(d)+"/"+titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    df_all.to_csv(os.path.join(path,'t='+t)+'.csv')